In [1]:
import sqlite3

# Define the SQLite database file name
db_filename = "example.db"

# Create (or connect to) the SQLite database
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Create a sample table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS sales (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL
    )
""")

# Insert sample data
cursor.execute("INSERT INTO sales (product, quantity, price) VALUES ('Widget', 10, 19.99)")
cursor.execute("INSERT INTO sales (product, quantity, price) VALUES ('Gadget', 5, 29.99)")

# Commit changes and close the connection
conn.commit()
conn.close()

print(f"SQLite database '{db_filename}' created successfully.")


SQLite database 'example.db' created successfully.


In [1]:
# backend.py (Back-end)
import os
import sqlite3
import pandas as pd
import subprocess

# Constants
DB_PATH = "ishikawa_data.db"
CATEGORIES = ["Materials", "Methods", "Machines", "Manpower", "Environment", "Measurement", "Target"]

def initialize_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS ishikawa_mapping (
                    column_name TEXT PRIMARY KEY,
                    category TEXT)''')
    conn.commit()
    conn.close()

def get_csv_columns(file_path):
    try:
        df = pd.read_csv(file_path)
        return list(df.columns)
    except FileNotFoundError:
        raise FileNotFoundError("CSV file not found.")
    except pd.errors.ParserError:
        raise ValueError("Error parsing CSV file. Check format.")
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}")

def process_data(file_path, mappings, target_column):
    try:
        df = pd.read_csv(file_path)

        # Map columns (handling missing mappings)
        df['Ishikawa'] = df.columns.map(lambda col: mappings.get(col) if col in mappings else "Uncategorized")

        # Check for uncategorized columns:
        if df['Ishikawa'].eq("Uncategorized").any():
            uncategorized_cols = df.columns[df['Ishikawa'] == "Uncategorized"].tolist()
            raise ValueError(f"The following columns are not categorized: {', '.join(uncategorized_cols)}")

        # Target column handling
        if target_column not in df.columns:
            raise ValueError(f"Target column '{target_column}' not found in CSV.")

        target_series = df.pop(target_column)  # Remove and return the target column
        df['Target'] = target_series  # Add it back as the last column

        summary = df.groupby('Ishikawa').agg(['count', 'sum', 'mean']).reset_index()
        summary.columns = ["_".join(col).strip() for col in summary.columns.values]
        summary_path = "summary.csv"
        summary.to_csv(summary_path, index=False)
        return summary_path

    except FileNotFoundError:
        raise FileNotFoundError("CSV file not found.")
    except pd.errors.ParserError:
        raise ValueError("Error parsing CSV file. Check format.")
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}")

def open_power_bi(summary_path, pbix_file):
    if os.path.exists(pbix_file):
        try:
            subprocess.run(["powerbi", pbix_file, summary_path], shell=True, check=True)
        except subprocess.CalledProcessError as e:
            raise Exception(f"Error opening Power BI: {e}")
        except FileNotFoundError:
            raise FileNotFoundError("Power BI executable not found. Is Power BI installed?")
    else:
        raise FileNotFoundError(f"Power BI template not found at: {pbix_file}")

# Initialize the database when the backend is imported
initialize_db()

In [3]:
# 1. Create a sample CSV file for testing
test_csv_path = "données_test3.csv"

In [4]:

# 3. Test process_data() with valid mappings
mappings = {
    'Column1': 'Materials',
    'Column2': 'Methods',
    'Column3': 'Machines',
    'TargetColumn': 'Target'  # Map the target column as well
}

In [12]:
try:
    summary_path = process_data(test_csv_path, mappings, 'TargetColumn')
    print(f"Summary CSV path: {summary_path}")
    summary_df = pd.read_csv(summary_path)  # Read the generated summary
    print("process_data() test with valid mappings passed!")
    print("Summary Dataframe:\n", summary_df) # Print the summary dataframe for inspection
    assert os.path.exists(summary_path), "Summary CSV not created"
    assert 'Target' in summary_df.columns, "Target column not in summary" # check if target column made it
    os.remove(summary_path) # Clean up the summary file after the test
except Exception as e:
    print(f"process_data() test with valid mappings failed: {e}")

process_data() test with valid mappings failed: An unexpected error occurred: Length of values (10) does not match length of index (1000)


In [10]:
def process_data(file_path, mappings, target_column):
    try:
        df = pd.read_csv(file_path)

        # 1. More Robust Mapping (Crucial):
        def map_category(col):  # Helper function
            category = mappings.get(col)
            if category is None:
                print(f"Warning: Column '{col}' not found in mappings. Using 'Uncategorized'.")  # Inform user
                return "Uncategorized"
            return category

        df['Ishikawa'] = df.columns.map(map_category) # Use the helper function

        # 2. Check for uncategorized columns AFTER mapping:
        uncategorized = df['Ishikawa'] == 'Uncategorized'
        if uncategorized.any():
            uncategorized_cols = df.columns[uncategorized].tolist()
            raise ValueError(f"The following columns are not categorized: {', '.join(uncategorized_cols)}")

        # 3. Target column handling (unchanged, but included for completeness):
        if target_column not in df.columns:
            raise ValueError(f"Target column '{target_column}' not found in CSV.")

        target_series = df.pop(target_column)  # Remove and return the target column
        df['Target'] = target_series  # Add it back as the last column

        # 4. Grouping and Summary (unchanged):
        summary = df.groupby('Ishikawa').agg(['count', 'sum', 'mean']).reset_index()
        summary.columns = ["_".join(col).strip() for col in summary.columns.values]
        summary_path = "summary.csv"
        summary.to_csv(summary_path, index=False)
        return summary_path

    except FileNotFoundError:
        raise FileNotFoundError("CSV file not found.")
    except pd.errors.ParserError:
        raise ValueError("Error parsing CSV file. Check format.")
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}")

In [ ]:
test =

NameError: name 'summary' is not defined

In [ ]:
# Test in Jupyter Notebook

import backend  # Import your backend module
import pandas as pd
import os



# 2. Test get_csv_columns()
try:
    columns = backend.get_csv_columns(test_csv_path)
    print(f"Columns from CSV: {columns}")
    assert columns == ['Column1', 'Column2', 'Column3', 'TargetColumn'], "Columns not retrieved correctly"
    print("get_csv_columns() test passed!")
except Exception as e:
    print(f"get_csv_columns() test failed: {e}")

# 3. Test process_data() with valid mappings
mappings = {
    'Column1': 'Materials',
    'Column2': 'Methods',
    'Column3': 'Machines',
    'TargetColumn': 'Target'  # Map the target column as well
}

try:
    summary_path = backend.process_data(test_csv_path, mappings, 'TargetColumn')
    print(f"Summary CSV path: {summary_path}")
    summary_df = pd.read_csv(summary_path)  # Read the generated summary
    print("process_data() test with valid mappings passed!")
    print("Summary Dataframe:\n", summary_df) # Print the summary dataframe for inspection
    assert os.path.exists(summary_path), "Summary CSV not created"
    assert 'Target' in summary_df.columns, "Target column not in summary" # check if target column made it
    os.remove(summary_path) # Clean up the summary file after the test
except Exception as e:
    print(f"process_data() test with valid mappings failed: {e}")


# 4. Test process_data() with missing mappings
mappings_missing = {
    'Column1': 'Materials',
    'Column2': 'Methods',
    'TargetColumn': 'Target' # Column3 is missing
}

try:
    backend.process_data(test_csv_path, mappings_missing, 'TargetColumn')
    print("process_data() test with missing mappings should have failed, but it passed!")
except ValueError as e: # Expect a ValueError
    print(f"process_data() test with missing mappings passed (expectedly): {e}")
except Exception as e:
    print(f"process_data() test with missing mappings failed with an unexpected error: {e}")


# 5. Test process_data() with invalid target column
try:
    backend.process_data(test_csv_path, mappings, 'NonExistentColumn')  # Invalid target column
    print("process_data() test with invalid target column should have failed, but it passed!")
except ValueError as e:  # Expect a ValueError
    print(f"process_data() test with invalid target column passed (expectedly): {e}")
except Exception as e:
    print(f"process_data() test with invalid target column failed with an unexpected error: {e}")

# 6. Test open_power_bi() (requires Power BI installed and template.pbix)
# Create a dummy summary file for the test
dummy_summary_path = "dummy_summary.csv"
dummy_df = pd.DataFrame({'Ishikawa': ['Materials', 'Methods'], 'Target_count':})
dummy_df.to_csv(dummy_summary_path, index=False)

try:
    backend.open_power_bi(dummy_summary_path, "template.pbix")  # Replace with your template path
    print("open_power_bi() test passed (assuming Power BI opened).")
    os.remove(dummy_summary_path) # Clean up the dummy file
except FileNotFoundError as e:
    print(f"open_power_bi() test failed: {e}")
except Exception as e:
    print(f"open_power_bi() test failed with an unexpected error: {e}")


# Cleanup the test CSV
os.remove(test_csv_path)

print("All tests completed.")

SyntaxError: invalid syntax (4092781129.py, line 68)

In [9]:
def process_data(file_path, mappings, target_column):
    try:
        df = pd.read_csv(file_path)
        print(f"DataFrame Columns: {df.columns}")  # Print DataFrame columns
        print(f"Mappings Dictionary: {mappings}")  # Print the mappings dictionary

        #... (rest of the process_data function)

SyntaxError: unexpected EOF while parsing (87226374.py, line 7)

In [2]:
import subprocess
from tkinter import filedialog

def open_powerbi_template():
    """Opens a file dialog to select a Power BI template file (.pbit) and 
    executes it using the `subprocess` module, which will launch Power BI Desktop.
    """
    file_path = filedialog.askopenfilename(filetypes=[("Power BI Template Files", "*.pbit")])
    if file_path:
        try:
            subprocess.run([file_path], shell=True)
        except Exception as e:
            print(f"Error opening Power BI template: {e}")

# Create the main window (if you don't have one already)
root = tk.Tk()
root.title("Open Power BI Template")

# Create a button to open the file dialog
open_button = tk.Button(root, text="Open Power BI Template", command=open_powerbi_template)
open_button.pack()

# Run the main loop
root.mainloop()

In [3]:
import subprocess
import os

def open_power_bi(pbix_file):
    """Opens the specified Power BI template file.

    Args:
        pbix_file (str): The path to the Power BI template file (.pbit).
    """
    if os.path.exists(pbix_file):
        try:
            subprocess.run(["powerbi", pbix_file], shell=True, check=True)
        except subprocess.CalledProcessError as e:
            raise Exception(f"Error opening Power BI: {e}")
        except FileNotFoundError:
            raise FileNotFoundError("Power BI executable not found. Is Power BI installed?")
    else:
        raise FileNotFoundError(f"Power BI template not found at: {pbix_file}")

In [ ]:
open_power_bi()